In [1]:
import boto3
import sagemaker
from sagemaker import get_execution_role
sess = boto3.Session()
sm = sess.client('sagemaker')
role = sagemaker.get_execution_role()

In [2]:
# Replace these values with your specific ones
docker_image_inference = '373770620841.dkr.ecr.ap-southeast-2.amazonaws.com/chatgptdetector-inference:chatgptdetector_image'
endpoint_name = 'chatgpt-detector-endpoint'

# Create a SageMaker client
sagemaker = boto3.client('sagemaker', region_name='ap-southeast-2')

# Create the model
model_name = 'chatgptdetector'
model_data = 's3://chatgptdetector/chatgptdetector/latest/model/model.tar.gz'


primary_container = {
    'Image': docker_image_inference,
    'ModelDataUrl': model_data,
}

create_model_response = sagemaker.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer=primary_container,
)

In [3]:
# Create the endpoint configuration
endpoint_config_name = 'chatgpt-detector-endpoint'
create_endpoint_config_response = sagemaker.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': 'AllTraffic',
            'ModelName': model_name,
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.m4.xlarge',  # Replace with your desired instance type
        },
    ],
)

In [4]:
# Create the endpoint
log_group_name = '/aws/sagemaker/chatgptdetector-endpoint'  # Replace with your desired log group name
# Create the CloudWatch log group
logs = boto3.client('logs', region_name='ap-southeast-2')  # Replace with your desired region
logs.create_log_group(logGroupName=log_group_name)

ResourceAlreadyExistsException: An error occurred (ResourceAlreadyExistsException) when calling the CreateLogGroup operation: The specified log group already exists

In [6]:
create_endpoint_response = sagemaker.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
    Tags=[
        {
            'Key': 'log_group',
            'Value': log_group_name
        },
    ]
)

print("Endpoint creation process started. This may take several minutes.")
print("Please check the AWS SageMaker console for endpoint status.")

Endpoint creation process started. This may take several minutes.
Please check the AWS SageMaker console for endpoint status.


In [15]:
import boto3
import json


# Define the input text for prediction
input_text = "This is a testinput"

# Create the request payload
payload = json.dumps({"input_text": input_text}).encode()

sample_data = payload

runtime = boto3.client('runtime.sagemaker')
response = runtime.invoke_endpoint(EndpointName='chatgpt-detector-endpoint',
                                   ContentType='application/json',
                                   Body=sample_data)
print(response['Body'].read())

b'{"input_text":"This is a testinput","predicted_class":0}\n'


In [16]:
response['Body']